In [68]:

from google.cloud import bigquery
from google.cloud import storage


GCP_PROJECT = 'kafka-408805'
BIGQUERY_DATASET = 'airbnb'
GCS_BUCKET = "kafka_airbnb"
GCS_BLOB_PREFIX = "topics"
gsc_client = storage.Client()
blobs = gsc_client.list_blobs(GCS_BUCKET, prefix='topics/', delimiter='/')
for blob in blobs:
    print(blob.name)
for blob_prefix in blobs.prefixes:
    topic = blob_prefix.split('/')[1]
    table_name = topic.split('.')[-1]
    table_id = f"{GCP_PROJECT}.{BIGQUERY_DATASET}.{table_name}"
    external_source_format = "AVRO"
    external_config = bigquery.ExternalConfig(external_source_format)
    exceptions = []
    bq_client = bigquery.Client(project=GCP_PROJECT)
    topic = blob_prefix.split('/')[1]
    table_name = topic.split('.')[-1]
    table_id = f"{GCP_PROJECT}.{BIGQUERY_DATASET}.{table_name}"
    r = None
    try:
        r = bq_client.get_table(table_id)
    except:
        pass
    if r:
        continue
    print(f'Creating table {table_id}...')
    external_config.source_uris = f'gs://kafka_airbnb/{blob_prefix}*'
    table = bigquery.Table(table_id)
    table.external_data_configuration = external_config
    bq_client.create_table(table)
    

Creating table kafka-408805.airbnb.nsw_lga_suburb...


Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/kafka-408805/datasets/airbnb/tables?prettyPrint=false: Access Denied: BigQuery BigQuery: Permission denied while globbing file pattern. tupham-local@kafka-408805.iam.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist). Please make sure gs://topics/airbnb.airbnb_raw.nsw_lga_suburb/* is accessible via appropriate IAM roles, e.g. Storage Object Viewer or Storage Object Creator.

In [63]:
blob_prefix.split('/')[1]


'airbnb.airbnb_raw.nsw_lga_suburb'